## Steps:
- load text
- Create embeddings
- Save as json in bucket
- Create index with above bucket
- Create endpoint
- Deploy index to endpoint



In [4]:
# !pip install --upgrade google-cloud-aiplatform google-cloud-storage 'google-cloud-bigquery[pandas]'

In [2]:
# # Restart kernel after installs so that your environment can access the new packages
# import IPython
# import time

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [1]:
# get project ID
PROJECT_ID = 'ecg-ai-416210'

In [4]:
LOCATION = 'europe-west1'

### Enable APIs

Run the following to enable APIs for Compute Engine, Vertex AI, Cloud Storage and BigQuery with this Google Cloud project.

In [8]:
# gcloud auth application-default login

In [9]:
# ! gcloud services enable compute.googleapis.com aiplatform.googleapis.com storage.googleapis.com bigquery.googleapis.com --project {PROJECT_ID}

## Data preparation

In [10]:
# load the BQ Table into a Pandas Dataframe
import pandas as pd
from google.cloud import bigquery

QUESTIONS_SIZE = 1000

bq_client = bigquery.Client(project=PROJECT_ID)
QUERY_TEMPLATE = """
        SELECT distinct q.id, q.title
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions`
        where Score > 0 ORDER BY View_Count desc) AS q
        LIMIT {limit} ;
        """
query = QUERY_TEMPLATE.format(limit=QUESTIONS_SIZE)
query_job = bq_client.query(query)
rows = query_job.result()
df = rows.to_dataframe()

# examine the data
df.head()

,id,title
0,64232071,gh-pages script cannot commit .nojekyll to GitHub
1,64276234,WARNING:tensorflow:Can save best model only wi...
2,64183423,Cumulative (running) sum of field Django and M...
3,64392487,Add space between cells in RecyclerView?
4,64285237,Ansbile way of having multiple MAILTO environm...


## Call the API to generate embeddings

In [17]:
# init the vertexai package
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [18]:
# Load the text embeddings model
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

By default, the text embeddings API has a "request per minute" quota set to 60 for new Cloud projects and 600 for projects with usage history (see Quotas and limits to check the latest quota value for base_model:textembedding-gecko). So, rather than using the function directly, you may want to define a wrapper like below to limit under 10 calls per second, and pass 5 texts each time.

In [19]:
import time
import tqdm  # to show a progress bar

# get embeddings for a list of texts
BATCH_SIZE = 5


def get_embeddings_wrapper(texts):
    embs = []
    for i in tqdm.tqdm(range(0, len(texts), BATCH_SIZE)):
        time.sleep(1)  # to avoid the quota error
        result = model.get_embeddings(texts[i : i + BATCH_SIZE])
        embs = embs + [e.values for e in result]
    return embs

In [20]:
# get embeddings for the question titles and add them as "embedding" column
df = df.assign(embedding=get_embeddings_wrapper(list(df.title)))
df.head()

100%|██████████| 200/200 [04:12<00:00,  1.26s/it]


,id,title,embedding
0,64232071,gh-pages script cannot commit .nojekyll to GitHub,"[0.014327898621559143, -0.01347998809069395, -..."
1,64276234,WARNING:tensorflow:Can save best model only wi...,"[0.016726359724998474, -0.026153581216931343, ..."
2,64183423,Cumulative (running) sum of field Django and M...,"[-0.025510864332318306, 0.0010844580829143524,..."
3,64392487,Add space between cells in RecyclerView?,"[-0.016930606216192245, -0.010796603746712208,..."
4,64285237,Ansbile way of having multiple MAILTO environm...,"[-0.0006941449828445911, -0.017993779852986336..."


In [25]:
len(df['embedding'][0])

768

In [26]:
len(df['embedding'][2])

768

## Save the embeddings in a JSON file

In [27]:
# save id and embedding as a json file
jsonl_string = df[["id", "embedding"]].to_json(orient="records", lines=True)
with open("questions.json", "w") as f:
    f.write(jsonl_string)

# show the first few lines of the json file
# ! head -n 3 questions.json

create a new Cloud Storage bucket and copy the file to it.

In [28]:
BUCKET_URI = f"gs://{PROJECT_ID}-text-embeddings"
! gsutil mb -l $LOCATION -p {PROJECT_ID} {BUCKET_URI}
! gsutil cp questions.json {BUCKET_URI}

Creating gs://ecg-ai-416210-text-embeddings/...
Copying file://questions.json [Content-Type=application/json]...
/ [1 files][  9.8 MiB/  9.8 MiB]                                                
Operation completed over 1 objects/9.8 MiB.                                      


## Creat an index

In [29]:
# init the aiplatform package
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

#### The parameters for creating index

- `contents_delta_uri`: The URI of Cloud Storage directory where you stored the embedding JSON files
- `dimensions`: Dimension size of each embedding. In this case, it is 768 as we are using the embeddings from the Text Embeddings API.
- `approximate_neighbors_count`: how many similar items we want to retrieve in typical cases
- `distance_measure_type`: what metrics to measure distance/similarity between embeddings. In this case it's `DOT_PRODUCT_DISTANCE`

See [the document](https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index) for more details on creating Index and the parameters.

#### Batch Update or Streaming Update?
There are two types of index: Index for *Batch Update* (used in this tutorial) and Index for *Streaming Updates*. The Batch Update index can be updated with a batch process whereas the Streaming Update index can be updated in real-time. The latter one is more suited for use cases where you want to add or update each embeddings in the index more often, and crucial to serve with the latest embeddings, such as e-commerce product search.


In [ ]:
# create index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"embvs-tutorial-index",
    contents_delta_uri=BUCKET_URI,
    dimensions=768,
    approximate_neighbors_count=20,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
)

By calling the `create_tree_ah_index` function, it starts building an Index. This will take under a few minutes if the dataset is small, otherwise about 50 minutes or more depending on the size of the dataset. You can check status of the index creation on [the Vector Search Console > INDEXES tab](https://console.cloud.google.com/vertex-ai/matching-engine/indexes).

### Create Index Endpoint and deploy the Index

To use the Index, you need to create an [Index Endpoint](https://cloud.google.com/vertex-ai/docs/vector-search/deploy-index-public). It works as a server instance accepting query requests for your Index.

In [33]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"embvs-tutorial-index-endpoint",
    public_endpoint_enabled=True,
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/500033913879/locations/europe-west1/indexEndpoints/3653061412588093440/operations/7651199601751359488
MatchingEngineIndexEndpoint created. Resource name: projects/500033913879/locations/europe-west1/indexEndpoints/3653061412588093440
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/500033913879/locations/europe-west1/indexEndpoints/3653061412588093440')


This tutorial utilizes a [Public Endpoint](https://cloud.google.com/vertex-ai/docs/vector-search/setup/setup#choose-endpoint) and does not support [Virtual Private Cloud (VPC)](https://cloud.google.com/vpc/docs/private-services-access). Unless you have a specific requirement for VPC, we recommend using a Public Endpoint. Despite the term "public" in its name, it does not imply open access to the public internet. Rather, it functions like other endpoints in Vertex AI services, which are secured by default through IAM. Without explicit IAM permissions, as we have previously established, no one can access the endpoint.

In [35]:
DEPLOYED_INDEX_ID = f"embvs_tutorial_deployed"
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

## Test embedding by query

In [36]:
test_embeddings = get_embeddings_wrapper(["How to read JSON with Python?"])

100%|██████████| 1/1 [00:01<00:00,  1.22s/it]


In [37]:
# Test query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=20,
)

# show the result
import numpy as np

for idx, neighbor in enumerate(response[0]):
    id = np.int64(neighbor.id)
    similar = df.query("id == @id", engine="python")
    print(f"{neighbor.distance:.4f} {similar.title.values[0]}")

0.7509 How to partially deserialise a JSON object?
0.7172 JSON parse reviver
0.6936 Basic request to mongodb with pymongo
0.6876 How to know Multiple TLS versions of the host using python?
0.6838 How to include OPENJSON in View?
0.6805 Not able to join a string properly in python
0.6802 How do I pivot this dataframe in Pandas with duplicate keys?
0.6796 Is there a way to fix this "TypeError: express.json is not a function" when testing Express with Jest?
0.6794 How to sort a nested dictionary in Python 2 times?
0.6788 How to add JSON request and response examples in Swagger (OpenApi)?
0.6781 Why is Pipenv not picking up my Pyenv versions?
0.6586 How can I merge a Pandas dataframes based on a substring from one of the columns?
0.6564 Python : How can I check if the content of one entire column of a Dataframe is empty?
0.6563 How to show json external api in input field in Angular 9?
0.6558 Acces array variables from a json file in Unity
0.6557 Django: How to filter model objects after p

#  Get an existing Index/Enpoints

In [38]:
my_index_id = "3642550081426554880"  # @param {type:"string"}
my_index = aiplatform.MatchingEngineIndex(my_index_id)

In [39]:
my_index

resource name: projects/500033913879/locations/europe-west1/indexes/3642550081426554880

In [40]:
my_index_endpoint_id = "7526157092126720000"  # @param {type:"string"}
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint_id)

## Cleaning up

In [41]:
# wait for a confirmation
# input("Press Enter to delete Index Endpoint, Index and Cloud Storage bucket:")

# # delete Index Endpoint
# my_index_endpoint.undeploy_all()
# my_index_endpoint.delete(force=True)

# # delete Index
# my_index.delete()

# delete Cloud Storage bucket
! gsutil rm -r {BUCKET_URI}

Removing gs://ecg-ai-416210-text-embeddings/questions.json#1711531973811442...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Removing gs://ecg-ai-416210-text-embeddings/...
